LLM based Reranking

In [ ]:
!pip install pyterrier-dr
!pip install pyterrier-t5

In [6]:
import pyterrier as pt


In [7]:
import pyterrier_alpha as pta
import pyterrier_dr as pyterrier_dr
model = pyterrier_dr.TasB()

index = pta.Artifact.from_hf('macavaney/msmarco-passage.tasb.flex')

print(index)

tasb = model >> index.np_retriever()


FlexIndex('/home/mandeep.rathee/.pyterrier/artifacts/56ec8babfb09acd89e01a803b5f24cbddc58531bf9ed1b66c4d1ac0c6b6a8780')


In [8]:
sparse_index = pt.Artifact.from_hf('macavaney/msmarco-passage.terrier')

print(sparse_index)

print(sparse_index.path)

sparse_index = pt.IndexFactory.of(f"{sparse_index.path}")

bm25 = pt.terrier.Retriever(sparse_index, wmodel="BM25")

TerrierIndex('/home/mandeep.rathee/.pyterrier/artifacts/7ead118630437940852142386f67ab62123a6ce372bb4b8cf12b06a76c8ccc25')
/home/mandeep.rathee/.pyterrier/artifacts/7ead118630437940852142386f67ab62123a6ce372bb4b8cf12b06a76c8ccc25


Java started (triggered by IndexFactory.of) and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


22:03:52.461 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [ ]:
from pyterrier.measures import *
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
pt.Experiment(
    [bm25 % 100,
    tasb % 100],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@100],
    names=['BM25', 'TaSB']
)

NumpyRetriever scoring: 100%|██████████| 2159/2159 [01:04<00:00, 33.53docbatch/s]


,name,AP(rel=2),R(rel=2)@1000,nDCG
0,BM25,0.232201,0.488365,0.445331
1,TaSB,0.405695,0.609619,0.581563


In [2]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker

/home/mandeep.rathee/IR Lectures/Exercise/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
monoT5 = MonoT5ReRanker(batch_size=16) 
duoT5 = DuoT5ReRanker() 

In [10]:
corpus =  pt.get_dataset('irds:msmarco-passage')
mono_pipeline = bm25 >> pt.text.get_text(corpus, "text") >> monoT5
duo_pipeline = mono_pipeline % 5 >> duoT5 # apply a rank cutoff of 5 from monoT5 since duoT5 is too costly to run over the full result list



In [14]:
monoT5_retrieval_result = mono_pipeline.search('Introduction to Information Retrieval')

[INFO] [starting] building docstore
[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /home/mandeep.rathee/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
                                                                                
                                                                                                                            [INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: [05:27] [1.06GB] [3.23MB/s]
[INFO] [starting] fixing encoding                                               
                                                                              

In [15]:
print(monoT5_retrieval_result)

    qid    docid    docno                                  query  \
1     1  8772671  8772671  Introduction to Information Retrieval   
337   1  1404848  1404848  Introduction to Information Retrieval   
95    1  8623714  8623714  Introduction to Information Retrieval   
217   1  1926200  1926200  Introduction to Information Retrieval   
225   1  5245411  5245411  Introduction to Information Retrieval   
..   ..      ...      ...                                    ...   
666   1  3558308  3558308  Introduction to Information Retrieval   
539   1  1049356  1049356  Introduction to Information Retrieval   
868   1  4127512  4127512  Introduction to Information Retrieval   
389   1  1610854  1610854  Introduction to Information Retrieval   
321   1  3590878  3590878  Introduction to Information Retrieval   

                                                  text      score  rank  
1    Introduction to Information Retrieval. This is...  -0.002678     0  
337  Information retrieval (IR) is 

In [19]:
from pyterrier.measures import *

ranker = pt.text.get_text(corpus, "text") >> monoT5

dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
pt.Experiment(
    [bm25 % 100,
    bm25 % 100 >> ranker,
    bm25 % 100 >> ranker % 5 >> duoT5],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG@10,nDCG@100, MAP(rel=2), R(rel=2)@100],
    names=['BM25', 'BM25>>MonoT5', 'BM25>>MonoT5>>DuoT5']
)

duoT5: 100%|██████████| 43/43 [04:09<00:00,  5.80s/queries]


,name,nDCG@10,nDCG@100,AP(rel=2),R(rel=2)@100
0,BM25,0.479540,0.487529,0.232201,0.488365
1,BM25>>MonoT5,0.699030,0.563323,0.358988,0.488365
2,BM25>>MonoT5>>DuoT5,0.512927,0.239960,0.152704,0.164331
